In [3]:
import pandas as pd
from functions import assembleDf, epochElo, epochsElo, epochG, epochsG,PlayerElo
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time

In [4]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

In [5]:
singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_4546/3293468424.py:1: DtypeWarning: Columns (3,9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'],


In [6]:
playerClasses, ratingsHistory = epochsElo(matches)

In [7]:
ratingsHistory_df = assembleDf(ratingsHistory)

In [8]:
singles_matches

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1877-540,Wimbledon,Grass,32,G,1877-07-09,19,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1877-540,Wimbledon,Grass,32,G,1877-07-09,21,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1877-540,Wimbledon,Grass,32,G,1877-07-09,20,113987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1877-540,Wimbledon,Grass,32,G,1877-07-09,18,113963,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1877-540,Wimbledon,Grass,32,G,1877-07-09,17,113999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876973,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,104,207987,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,716.0,31.0,966.0,12.0
876974,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,103,144932,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,799.0,23.0,1117.0,8.0
876975,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,102,209955,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876976,2022-M-ITF-TUN-63A-2022,M15 Monastir,Hard,32,15,2022-12-26,205,209079,NaN,JE,...,NaN,NaN,NaN,NaN,NaN,NaN,1125.0,7.0,1548.0,2.0


In [9]:
# fill in missing values with last seen rating.  
ratingsHistory_df = ratingsHistory_df.ffill(axis=0).fillna(1500)

In [10]:
max(ratingsHistory_df.index[ratingsHistory_df.index<'1880-07-08'])

Timestamp('1879-07-09 00:00:00')

In [11]:
# def winProb(winner_id,loser_id,tourney_date)

SyntaxError: invalid syntax (372932685.py, line 1)

In [12]:
type(ratingsHistory_df.index[10])

pandas._libs.tslibs.timestamps.Timestamp

In [20]:
max(ratingsHistory_df.index[ratingsHistory_df.index<='2020-08-09'])

Timestamp('2020-06-04 00:00:00')

In [27]:
def get_recent_rating_wp(ratingsHistory_df,tourney_date,winner_id,loser_id):
    # if not type(tourney_date) == pandas._libs.tslibs.timestamps.Timestamp:
    #     tourney_date = datetime.strptime(tourney_date, '%Y-%m-%d').date()

    timestamp = max(ratingsHistory_df.index[ratingsHistory_df.index<=tourney_date])
    winner_rating, loser_rating = ratingsHistory_df.loc[timestamp,[winner_id,loser_id]]
    wp = 1/(1 + 10**((loser_rating-winner_rating)/400))
    return winner_rating, loser_rating, wp

In [ ]:
get_recent_rating_wp(ratingsHistory_df,'1878-07-09',113987,114149)

In [28]:
get_recent_rating_wp(ratingsHistory_df,'1878-07-09',113987,114149)

(1532.0, 1508.0, 0.5344839447268315)

In [ ]:
matches

In [ ]:
matches

In [ ]:
singles_matches.shape,matches.shape

In [ ]:
singles_matches[['winner_elo','loser_elo','predicted_wins']] = matches[['tourney_date','winner_id','loser_id']].apply(lambda x: 
get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], x['winner_id'], x['loser_id']),axis=0)

In [ ]:
ratingsHistory_df